In [5]:
import requests
import pandas as pd
import json
import numpy as np

In [2]:
supply_r = requests.get('https://peridash.ml/api/supply-areas/')
measurements_r = requests.get('https://peridash.ml/api/measurements/')
epsas_r = requests.get('https://peridash.ml/api/epsas/')

In [3]:
measurements_df = pd.read_json(measurements_r.text)
epsas_df = pd.read_json(epsas_r.text)
df = pd.merge(measurements_df, epsas_df, left_on='epsa', right_on='url')

In [4]:
supply_json = json.loads(supply_r.text)

epsa_jsons = []
for feature in supply_json['features']:
    epsa_json = dict(
        type='FeatureCollection',
        features=[feature]
    )
    epsa_jsons.append(epsa_json)
    
with open('points.json', encoding='utf8') as f:
    points_json = json.load(f)

In [6]:
val_year = 2017

min_value = min(list(df[df.year == val_year].ind8))

min_ranges = [0.0, 0.35, 0.5, 0.6, 0.7]
max_ranges = [0.35, 0.5, 0.6, 0.7, 1.1]
min_colors = [[220,220,220],[106,137,247],[90,120,245],[70,100,245],[40,60,190]]
max_colors = [[106,137,247],[90,120,245],[70,100,245],[40,60,190],[5,10,172]]

val_label = 'Indicador 8'
val_label2 = 'Indicador 9'
val_unit = '%'

def interpolate(c1, c2, f):
    r = c1[0] + (c2[0]-c1[0]) * f
    g = c1[1] + (c2[1]-c1[1]) * f
    b = c1[2] + (c2[2]-c1[2]) * f
    return [r,g,b]

def get_layer_code(layer_index):
    code = epsa_jsons[layer_index]['features'][0]['properties']['code']
    code = code.split('-')[0]
    return code

def get_layer_color(epsa_code, min_value):
    if epsa_code in list(df.code):
        percentage = df[(df.code==epsa_code)&(df.year==val_year)].ind8.iloc[0]
        if np.isnan(percentage):
            percentage = 0 
    else:
        percentage = 0
        
    factor = (percentage - min_value)/(100 - min_value) # Biased factor for more contrast
    
    color1 = min_colors[0]
    color2 = min_colors[1]
    real_factor = 0.0
    for range_min, range_max, c_min, c_max in zip(min_ranges, max_ranges, min_colors, max_colors):
        if range_min <= factor < range_max:
            color1 = c_min
            color2 = c_max
            real_factor = (factor - range_min) / (range_max - range_min)
    
    c = interpolate(color1, color2, real_factor)
    color_string = f'rgb({str(c[0])[:6]},{str(c[1])[:6]},{str(c[2])[:6]})'
    return(color_string)

def get_display_value(epsa_code):
    if epsa_code in list(df.code):
        val = df[(df.code==epsa_code)&(df.year==val_year)].ind8.iloc[0]
        if np.isnan(val):
            val = 0.0
    else:
        val = 0.0
    return val

def get_display_value2(epsa_code):
    if epsa_code in list(df.code):
        val = df[(df.code==epsa_code)&(df.year==val_year)].ind9.iloc[0]
        if np.isnan(val):
            val = 0.0
    else:
        val = 0.0
    return val

layers = [
    dict(
        sourcetype = 'geojson',
        source = epsa_jsons[k],
#         below = "water",
        type = 'fill',   
        color = get_layer_color(get_layer_code(k), min_value),
        opacity = 0.8
    ) for k in range(len(epsa_jsons))
]

p_lats = []
p_lons = []
p_texts = []

for code in list(points_json.keys()):
    epsa_p = points_json[code]
    n = len(epsa_p['lats'])
    p_lats += epsa_p['lats']
    p_lons += epsa_p['lons']
    
    clean_code = code.split('-')[0]
    value = get_display_value(clean_code)
    value2= get_display_value2(clean_code)
    display_text = f'{clean_code}<br>{val_label}: {value} ({val_unit})<br>{val_label2}: {value2} ({val_unit})'
    p_texts += [display_text for k in range(n)]
    
scatter_dict = dict(
    type = 'scattermapbox',
    mode = 'markers',
    
    lat = p_lats, 
    lon = p_lons,
    text = p_texts,
    
    marker = dict(
        size = 50,
        opacity = 0,
        color= 'rgb(37,52,148)'
    ),
    showlegend = False,
    hoverinfo = 'text',
)

mapbox_access_token = 'pk.eyJ1Ijoic2VyZ2lvLWNodW1hY2Vyby1maSIsImEiOiJjamswOTUzeHkwMDk0M3dvNnJoeTByZGlpIn0.3mmjpLwDrIUcdJTowlCd1A'

layout = dict(
    title = 'Indicadores de Cobertura',
    font = dict(family='Balto'),
    autosize = False,
    width = 1200,
    height = 700,
    hovermode = 'closest',

    mapbox = dict(
        accesstoken = mapbox_access_token,
        layers = layers,
        bearing = 0,
        center = dict( 
            lat = -17.610907366555434, 
            lon = -63.13396632812757,
        ),
        pitch = 0,
        zoom = 8,
        style = 'light'
    ) 
)

fig = dict(data=[scatter_dict], layout=layout)

import plotly.plotly as py
py.iplot(fig, filename='mapa-coberturas-new')